# 03_clip_embeddings.ipynb
## Pruebas de CLIP Embeddings y Espacio Vectorial Compartido

**CRÍTICO**: Valida que imagen y texto estén en el MISMO espacio vectorial

Esto es el CORAZÓN del "Enfoque 3: RAG Multimodal Verdadero" de Clase 17

In [ ]:
import sys
from pathlib import Path
import numpy as np
import json

sys.path.insert(0, str(Path.cwd()))

from src.embeddings import CLIPEncoder
from src.utils.config import EXCEL_IMAGES_DIR, PDF_IMAGES_DIR
from src.utils.logger import get_logger

logger = get_logger(__name__)

## 1. Inicializar CLIP Encoder

In [ ]:
print("Cargando modelo CLIP pre-entrenado...")
encoder = CLIPEncoder()
print("✅ CLIP encoder listo")

# Información del modelo
print(f"\nInformación del modelo:")
print(f"  - Modelo: openai/clip-vit-base-patch32")
print(f"  - Dimensionalidad: 512")
print(f"  - Pre-entrenado en: ~400M pares imagen-texto")

## 2. Codificar Imagen

In [ ]:
# Obtener imagen de prueba
excel_images = list(EXCEL_IMAGES_DIR.glob("*.png"))

if excel_images:
    test_image = excel_images[0]
    print(f"Codificando imagen: {test_image.name}")
    
    # Encodear imagen
    image_embedding = encoder.encode_image(str(test_image))
    
    if image_embedding is not None:
        print(f"\n✅ Embedding generado:")
        print(f"   - Dimensiones: {image_embedding.shape}")
        print(f"   - Tipo: {type(image_embedding)}")
        print(f"   - Primeros 5 valores: {image_embedding[:5]}")
        print(f"   - Norma (debe ser ~1.0): {np.linalg.norm(image_embedding):.4f}")
    else:
        print("❌ Error al codificar imagen")
else:
    print("⚠️ No hay imágenes. Ejecuta notebook 01.")

## 3. Codificar Texto

In [ ]:
# Textos de prueba
test_texts = [
    "¿Cuál es el total de la liquidación?",
    "Liquidación de sueldo y gratificaciones",
    "Tabla con montos y conceptos de pago",
    "Esto es un documento completamente diferente sobre gatos"
]

text_embeddings = {}

for text in test_texts:
    print(f"\nCodificando: '{text[:50]}...'")
    embedding = encoder.encode_text(text)
    
    if embedding is not None:
        text_embeddings[text] = embedding
        print(f"  ✅ Dimensiones: {embedding.shape}")
        print(f"  ✅ Norma: {np.linalg.norm(embedding):.4f}")

## 4. ⭐ VERIFICAR ESPACIO COMPARTIDO

In [ ]:
# ESTO ES CRÍTICO: Verificar que imagen y texto pueden compararse

print("="*60)
print("VERIFICACIÓN ESPACIO VECTORIAL COMPARTIDO")
print("="*60)

if 'image_embedding' in locals() and text_embeddings:
    # Verificar dimensiones
    img_dim = image_embedding.shape[0]
    text_dim = text_embeddings[test_texts[0]].shape[0]
    
    print(f"\n1. Dimensionalidad:")
    print(f"   Imagen: {img_dim}")
    print(f"   Texto: {text_dim}")
    print(f"   ¿Iguales? {'✅ SÍ' if img_dim == text_dim else '❌ NO'}")
    
    # Calcular similitud coseno
    print(f"\n2. Similitud Coseno (Imagen vs Textos):")
    print(f"   {'Texto':<40} | {'Similitud':>10}")
    print(f"   {'-'*40}-{'-'*10}")
    
    for text, embedding in text_embeddings.items():
        similarity = np.dot(image_embedding, embedding)
        print(f"   {text[:39]:<40} | {similarity:>10.4f}")
    
    print(f"\n3. Interpretación:")
    print(f"   - Valores cerca de 1.0 = SIMILARES")
    print(f"   - Valores cerca de 0.0 = DIFERENTES")
    print(f"   - Valores cerca de -1.0 = OPUESTOS")
    
    print(f"\n✅ CONCLUSIÓN: Imagen y texto están en MISMO espacio\n")
else:
    print("❌ Faltan embeddings para comparación")

print("="*60)

## 5. Método oficial: verify_shared_space()

In [ ]:
# Usar método oficial del encoder
if excel_images:
    test_image = excel_images[0]
    test_text = test_texts[0]
    
    print(f"Verificando espacio compartido...\n")
    verification = encoder.verify_shared_space(str(test_image), test_text)
    
    print(f"Resultado verificación:")
    print(json.dumps(verification, indent=2))
    
    if verification.get('shared_space'):
        print(f"\n✅ ÉXITO: Espacio vectorial compartido verificado")

## 6. Batch Processing: Múltiples Imágenes

In [ ]:
# Procesar múltiples imágenes
all_images = list(EXCEL_IMAGES_DIR.glob("*.png")) + list(PDF_IMAGES_DIR.glob("*.png"))
image_paths = [str(img) for img in all_images[:5]]  # Primeras 5

if image_paths:
    print(f"Procesando {len(image_paths)} imágenes en batch...\n")
    
    embeddings = encoder.batch_encode_images(image_paths)
    
    print(f"\nResultados:")
    print(f"  Imágenes procesadas: {len(embeddings)}")
    
    for path, data in list(embeddings.items())[:3]:
        print(f"\n  - {Path(path).name}")
        print(f"    Dimensión: {np.array(data['embedding']).shape}")
        print(f"    Tipo: {data['type']}")

## 7. Batch Processing: Múltiples Textos

In [ ]:
# Procesar múltiples textos
test_docs = [
    "Liquidación de abril de 2024 con bonificación",
    "Pago de sueldo base y AFP",
    "Resumen de ingresos y descuentos",
    "Tabla de conceptos de pago con montos",
    "Documento de recursos humanos"
]

print(f"Procesando {len(test_docs)} textos en batch...\n")

text_embeddings_batch = encoder.batch_encode_texts(test_docs)

print(f"\nResultados:")
print(f"  Textos procesados: {len(text_embeddings_batch)}")

for key, data in list(text_embeddings_batch.items())[:3]:
    print(f"\n  - {key}")
    print(f"    Contenido: {data['content']}")
    print(f"    Dimensión: {np.array(data['embedding']).shape}")

## 8. Visualización: t-SNE del Espacio Vectorial

In [ ]:
# Visualizar el espacio vectorial con t-SNE
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt

if embeddings and text_embeddings_batch:
    print("Reduciendo dimensionalidad con t-SNE...")
    
    # Recolectar todos los embeddings
    all_embeddings = []
    labels = []
    colors_list = []
    
    # Imágenes
    for path, data in list(embeddings.items())[:3]:
        all_embeddings.append(data['embedding'])
        labels.append(Path(path).name[:20])
        colors_list.append('red')
    
    # Textos
    for key, data in list(text_embeddings_batch.items())[:3]:
        all_embeddings.append(data['embedding'])
        labels.append(f"Text: {data['content'][:15]}")
        colors_list.append('blue')
    
    # Aplicar t-SNE
    embeddings_array = np.array(all_embeddings)
    tsne = TSNE(n_components=2, random_state=42, perplexity=min(5, len(all_embeddings)-1))
    embeddings_2d = tsne.fit_transform(embeddings_array)
    
    # Plotear
    plt.figure(figsize=(12, 8))
    
    for i, (x, y) in enumerate(embeddings_2d):
        plt.scatter(x, y, s=200, c=colors_list[i], alpha=0.7, edgecolors='black')
        plt.annotate(labels[i], (x, y), fontsize=8, ha='center')
    
    plt.title('Espacio Vectorial CLIP (t-SNE)\nRojo: Imágenes | Azul: Textos', fontsize=14)
    plt.xlabel('Dimensión 1 (t-SNE)')
    plt.ylabel('Dimensión 2 (t-SNE)')
    plt.grid(True, alpha=0.3)
    
    plt.savefig('clip_embeddings_tsne.png', dpi=150, bbox_inches='tight')
    plt.show()
    
    print("✅ Visualización guardada: clip_embeddings_tsne.png")
else:
    print("❌ No hay suficientes embeddings para visualizar")

## 9. Matriz de Similitud

In [ ]:
# Crear matriz de similitud imagen-texto
import seaborn as sns

if embeddings and text_embeddings_batch:
    print("Calculando matriz de similitud...\n")
    
    image_embs = [data['embedding'] for data in list(embeddings.values())[:3]]
    text_embs = [data['embedding'] for data in list(text_embeddings_batch.values())[:3]]
    image_labels = [Path(path).name[:15] for path in list(embeddings.keys())[:3]]
    text_labels = [data['content'][:15] for data in list(text_embeddings_batch.values())[:3]]
    
    # Calcular similitudes
    similarity_matrix = np.zeros((len(image_embs), len(text_embs)))
    
    for i, img_emb in enumerate(image_embs):
        for j, text_emb in enumerate(text_embs):
            similarity = np.dot(img_emb, text_emb)
            similarity_matrix[i, j] = similarity
    
    # Plotear
    plt.figure(figsize=(10, 6))
    sns.heatmap(similarity_matrix, annot=True, fmt='.3f', 
                xticklabels=text_labels, yticklabels=image_labels,
                cmap='RdYlGn', center=0, vmin=-1, vmax=1)
    plt.title('Matriz de Similitud: Imágenes vs Textos\n(Espacio Compartido CLIP)')
    plt.tight_layout()
    
    plt.savefig('clip_similarity_matrix.png', dpi=150, bbox_inches='tight')
    plt.show()
    
    print("✅ Matriz guardada: clip_similarity_matrix.png")

## 10. Guardar Embeddings para Indexación

In [ ]:
# Guardar embeddings en formato JSON
from src.embeddings import save_embeddings

if embeddings:
    print("Guardando embeddings de imágenes...")
    success = save_embeddings(embeddings, 'image_embeddings_test.json')
    if success:
        print("✅ Embeddings guardados")

if text_embeddings_batch:
    print("\nGuardando embeddings de textos...")
    success = save_embeddings(text_embeddings_batch, 'text_embeddings_test.json')
    if success:
        print("✅ Embeddings guardados")

## 11. Resumen y Conceptos

In [ ]:
print("="*70)
print("RESUMEN: CLIP EMBEDDINGS & ESPACIO MULTIMODAL")
print("="*70)

print(f"\n✅ LOGROS:")
print(f"   1. Cargar modelo CLIP pre-entrenado")
print(f"   2. Codificar imágenes → 512 dimensiones")
print(f"   3. Codificar textos → 512 dimensiones (MISMO espacio)")
print(f"   4. Calcular similitud entre imagen ↔ texto")
print(f"   5. Visualizar con t-SNE")
print(f"   6. Crear matriz de similitud")
print(f"   7. Guardar embeddings en JSON")

print(f"\n📚 CONCEPTOS CLASE 17:")
print(f"   ✅ Multimodalidad: Imagen + Texto en mismo espacio")
print(f"   ✅ CLIP: Modelo pre-entrenado OpenAI")
print(f"   ✅ Espacio Compartido: 512 dimensiones para ambos tipos")
print(f"   ✅ Similitud Coseno: Comparar imagen ↔ texto")
print(f"   ✅ RAG Multimodal: Base para retrieval")

print(f"\n🚀 PRÓXIMO PASO:")
print(f"   → Notebook 04: Agent Prototype (LangGraph + ChromaDB)")
print("="*70)